# Notebook as a Step using Notebooks Executor
The following sample shows how to use the notebook executor as part of a Vertex AI


## Libraries and Variables

In [ ]:
!which pip

In [ ]:
!pip install --upgrade google-cloud-aiplatform -q
!pip install --upgrade google-cloud-pipeline-components -q

In [ ]:
import kfp
import os

from datetime import datetime
from google.cloud import aiplatform
from google.cloud.aiplatform import pipeline_jobs
from kfp.v2 import compiler

In [ ]:
kfp.__version__

In [ ]:
# Variables
PROJECT_ID = '<YOUR-PROJECT-ID>'
REGION = 'us-central1' # This is where the KMS and Metastore resides from the project configuration

ROOT_PATH = ".."
PIPELINE_ROOT_PATH = f'gs://{PROJECT_ID}-vertex-root'
PACKAGE_PATH = 'notebook-as-a-step-sample-pipeline.json'

RUNNABLES_PATH = './runnables'
COMPONENT_YAML_PATH = os.path.join(ROOT_PATH, 'component.yaml')
if not os.path.isfile(COMPONENT_YAML_PATH):
    print(f'COMPONENT_YAML_PATH does not exist')

WORKING_BUCKET_NAME = f'{PROJECT_ID}-naas'
INPUT_NOTEBOOK_FILE = f'gs://{WORKING_BUCKET_NAME}/runnables/run_create_bucket.ipynb'
OUTPUT_NOTEBOOK_FOLDER = f'gs://{WORKING_BUCKET_NAME}/outputs'

INPUT_NOTEBOOK_FILE

In [ ]:
!gcloud config set project "{PROJECT_ID}"
!gcloud config list

In [ ]:
# The service account used by the Pipelines must have access to this bucket.
!gsutil ls "{PIPELINE_ROOT_PATH}" || gsutil mb -l "{REGION}" "{PIPELINE_ROOT_PATH}"
!gsutil cp -r "{RUNNABLES_PATH}/*" "gs://{WORKING_BUCKET_NAME}/runnables"
!gsutil ls "gs://{WORKING_BUCKET_NAME}/runnables"

## Prerequisites
You need to configure your project as detailed in https://cloud.google.com/vertex-ai/docs/pipelines/configure-project

In [ ]:
@kfp.dsl.pipeline(
    name="notebook-as-a-step-sample",
    pipeline_root=PIPELINE_ROOT_PATH)
def pipeline(
        project: str, 
        execution_id: str, 
        input_notebook_file:str, 
        output_notebook_folder:str,
        location:str,
        master_type:str,
        container_image_uri:str):
    
    execute_notebook_component = kfp.components.load_component_from_file(COMPONENT_YAML_PATH)
    execute_notebook_op = execute_notebook_component(
        project=project,
        execution_id=execution_id,
        input_notebook_file=input_notebook_file,
        output_notebook_folder=output_notebook_folder,
        location=location,
        master_type=master_type,
        container_image_uri=container_image_uri,
        parameters=f'PROJECT_ID={project},EXECUTION_ID={execution_id}'
    )

For additional details about:
- Google Cloud Notebook Executor template, see the [ExecutionTemplate API documentation](https://cloud.google.com/notebooks/docs/reference/rest/v1/ExecutionTemplate).
- Pipeline types, see [kfp.dsl.types](https://kubeflow-pipelines.readthedocs.io/en/latest/_modules/kfp/dsl/types.html).
- Notebook parameters, see [Papermill parameters](https://papermill.readthedocs.io/en/latest/usage-parameterize.html)

## Run pipeline

In [ ]:
NOW = datetime.now().strftime("%Y%m%d%H%M%S")
JOB_ID=f'naas-{NOW}'
EXECUTION_ID=f'naas_{NOW}'

compiler.Compiler().compile(
    pipeline_func=pipeline, package_path=PACKAGE_PATH)

pl = pipeline_jobs.PipelineJob(
    display_name='notebook-executor-pipeline',
    template_path=PACKAGE_PATH,
    job_id=JOB_ID,
    parameter_values={
        'project': PROJECT_ID,
        'execution_id': EXECUTION_ID,
        'input_notebook_file': INPUT_NOTEBOOK_FILE,
        'output_notebook_folder': OUTPUT_NOTEBOOK_FOLDER,
        'location': 'us-central1',
        'master_type': 'n1-standard-4',
        'container_image_uri': 'gcr.io/deeplearning-platform-release/base-cpu'
    },
)

pl.run()